In [3]:
#import tensorflow as tf
import numpy as np
import os
import cv2

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []

Not found stats file.
